# Linear Model: Make Predictions for Missing Orbit Semi-Major Axis

## Part 1: Data Processing

In [64]:
# import packages and data
import numpy as np
import pandas as pd
from sklearn import linear_model

# Orbit period, mass, radii, and orbital semi-major axis
gaia = pd.read_csv('../data/confirmed_transit.csv', comment='#')
gaia_new = gaia.loc[:,['pl_orbper','pl_bmassj','pl_radj','pl_orbsmax']]
gaia_np = gaia_new.to_numpy()
gaia_new.isna().sum()

pl_orbper       0
pl_bmassj       0
pl_radj         0
pl_orbsmax    570
dtype: int64

In [66]:
# Drop the Nah rows and save the Nah rows to a new dataframe for future use
gaia_new_nonah = gaia_new.dropna()
gaia_unknowns = pd.concat([gaia_new,gaia_new_nonah]).drop_duplicates(keep=False)
gaia_new_nonah = gaia_new_nonah.to_numpy()
gaia_unknowns_np = gaia_unknowns.to_numpy()
gaia_unknowns.head()

,pl_orbper,pl_bmassj,pl_radj,pl_orbsmax
2,41.685500,0.07000,0.230,NaN
7,1.508956,1.03000,1.490,NaN
38,1.742997,3.31000,1.414,NaN
66,4.256800,21.66000,1.010,NaN
81,0.854000,0.01447,0.141,NaN


## Part 2: Find the true relationship for Orbit Semi-Axis
Now use kfold_CV to determine the shape of the "true" relationship for orbit Semi-Axis. We consider all possible combinations.

In [6]:
# Helper function to compute the error between truth values and prediction
def compute_mse(truth_vec, predict_vec):
    return np.mean((truth_vec - predict_vec)**2)

In [38]:
# K folds of cross validation
def kfold_CV(data, col_names, inputs, output, k):
    n_data = data.shape[0]
    test_errors=[]
    
    for i in range(k):
        #split data into train and test
        test_ind = list(np.arange(int(n_data*i/k),int(n_data*(i+1)/k)))
        test_data = data[test_ind,:]
        train_ind = list(set(range(n_data)).difference(set(test_ind)))
        train_data = data[train_ind,:]
        
        # map inputs and output col into integer
        input_index = []
        for elem in inputs:
            input_index.append(col_names.index(elem))
        output_index = col_names.index(output)
        
        #fit model to training dataset
        lm = linear_model.LinearRegression()
        mod = lm.fit(train_data[:,input_index],train_data[:,output_index])
        
        #compute the testing error and add it to the list of testing errors
        test_preds = mod.predict(test_data[:,input_index])
        test_error = compute_mse(test_preds,test_data[:,output_index])
        test_errors.append(test_error)
    
    # Compute tehe cross-val error
    cross_val_error = np.mean(test_errors)
    return cross_val_error.astype(float)

In [51]:
# Column names
cols=['pl_orbper','pl_bmassj','pl_radj','pl_orbsmax']

# Find the smallest error -> the best relationship for describing orbit semi-axis
print("OP            :", kfold_CV(gaia_new_nonah,cols,['pl_orbper'],'pl_orbsmax',10))
print("MASS          :", kfold_CV(gaia_new_nonah,cols,['pl_bmassj'],'pl_orbsmax',10))
print("RADII         :", kfold_CV(gaia_new_nonah,cols,['pl_radj'],'pl_orbsmax',10))
print("OP+MASS       :", kfold_CV(gaia_new_nonah,cols,['pl_orbper','pl_bmassj'],'pl_orbsmax',10))
print("OP+RADII      :", kfold_CV(gaia_new_nonah,cols,['pl_orbper','pl_radj'],'pl_orbsmax',10))
print("MASS+RADII    :", kfold_CV(gaia_new_nonah,cols,['pl_bmassj','pl_radj'],'pl_orbsmax',10))
print("OP+MASS+RADII :", kfold_CV(gaia_new_nonah,cols,['pl_orbper','pl_bmassj','pl_radj'],'pl_orbsmax',10))

OP            : 0.023262732438269016
MASS          : 0.03310199549731842
RADII         : 0.032391992727619424
OP+MASS       : 0.02329911906411873
OP+RADII      : 0.022166508371421832
MASS+RADII    : 0.03211421662981265
OP+MASS+RADII : 0.022189014328411312


The lowest error come from the combination of orbital period and planet radii.

# Part 3: Determine the full model

In [60]:
# run the full model with orbit period and radii
lm_full = linear_model.LinearRegression()
mod_full = lm_full.fit(gaia_new_nonah[:,[0,2]],gaia_new_nonah[:,3])

# Predict the orbit major-axis
pl_orbsmax_nah = mod_full.predict(gaia_unknowns_np[:,[0,2]])

In [83]:
# Combine the regression results back to dataframe
m = gaia_unknowns['pl_orbsmax'].isna()
gaia_unknowns.loc[m,'pl_orbsmax'] = pl_orbsmax_nah
gaia_unknowns.head()

,pl_orbper,pl_bmassj,pl_radj,pl_orbsmax
2,41.685500,0.07000,0.230,0.149298
7,1.508956,1.03000,1.490,0.044130
38,1.742997,3.31000,1.414,0.047238
66,4.256800,21.66000,1.010,0.065633
81,0.854000,0.01447,0.141,0.092180


Now we have a prediction for Orbit Semi-Major Axis based on our best relationship!